# Data sanity check process

In this notebook, we will extract a dataset and the content of a `data_format_ref` file

we will run a data sanity check on this dataset based on the content of`data_format_ref`

In [1]:
import pandas as pd

import fedbiomed.common.data_tool.utils as utils
import fedbiomed.common.data_tool.multi_view_dataframe as multiview

from fedbiomed.common.data_tool.warning_logger import WarningReportLogger
from fedbiomed.common.data_tool.pre_processing_checker import PreProcessingChecker

In [2]:
# loading simple dataset

single_dataset_to_check = utils.load_tabular_datasets(r'pseudo_adni_mod_2.csv')

# loading `data_format_ref`

single_format_file_ref = utils.load_format_file_ref('single_format_file_ref_test3')




file found


In [2]:
# select only features in dataset that will be checked
pre_parsed_dataset_to_check = multiview.select_data_from_format_file_ref(single_dataset_to_check,
                                                                         single_format_file_ref)

pre_parsed_dataset_to_check

primary_key = multiview.search_primary_key(single_format_file_ref)

if primary_key is None:
    print('not a multi view file')
    
df_joined =  multiview.join_multi_view_dataset(pre_parsed_dataset_to_check, primary_key, False)
warning_report = WarningReportLogger(disclosure=3)

checker = PreProcessingChecker(single_format_file_ref, df_joined,
                               'dataFormat_file' ,
                               warning_report,
                              min_nb_samples=30)

#checker.update_views_features_name(new_views_name)
checker.check_all()

NameError: name 'single_dataset_to_check' is not defined

TODO : avoid warning when datatype is UNKNOWN

In [4]:
report = checker.get_warning_logger()
report

number of warnings: 3
Number of error: 0


{'N_SAMPLES_BELOW_THRESHOLD': [{'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'}],
 'INCORRECT_FORMAT_FILE': [{'view': '',
   'success': True,
   'feature': 'CDRSB.bl',
   'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'ADAS11.bl', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'MMSE.bl', 'msg': 'Test passed'},
  {'view': '',
   'success': True,
   'feature': 'RAVLT.immediate.bl',
   'msg': 'Test passed'},
  {'view': '',
   'success': True,
   'feature': 'RAVLT.learning.bl',
   'msg': 'Test passed'},
  {'view': '',
   'success': True,
   'feature': 'RAVLT.forgetting.bl',
   'msg': 'Test passed'},
  {'view': '',
   'success': True,
   'feature': 'Ventricles.bl',
   'msg': 'Test passed'},
  {'view': '',
   'success': True,
   'feature': 'Hippocampus.bl',
   'msg': 'Test passed'},
  {'view': '',
   'success': True,
   'feature': 'ABETA.MEDIAN.bl',
   'msg': 'Test passed'},
  {'view': '',
   'success': 

In [5]:
checker._warning_logger.raise_exception()

In [2]:
# loading dataset

multi_dataset_to_check = utils.load_tabular_datasets(r'demo_poc_data_wrapper/data/test7')

directory found


In [3]:
# loading `data_fromat_ref`

multi_format_file_ref = utils.load_format_file_ref('int_multi_view_format_test')

In [4]:
multi_format_file_ref

{'global_thresholds': {'min_nb_samples': None, 'min_nb_missing_samples': None},
 'file1': {'a': {'data_format': 'KEY',
   'data_type': 'DATETIME',
   'values': "<class 'numpy.datetime64'>",
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'e': {'data_format': 'KEY',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'time': {'data_format': 'KEY',
   'data_type': 'DATETIME',
   'values': "<class 'numpy.datetime64'>",
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None}}}

we wil extract

In [4]:
# select only features in dataset that will be checked
pre_parsed_dataset_to_check = multiview.select_data_from_format_file_ref(multi_dataset_to_check,
                                                                         multi_format_file_ref)

the dataset loaded is a dataset composed from multiple CSV files. To convert those dataset into a single one, we will do a join operation based on a primary key

let's  find this primary key:

In [5]:
primary_key = multiview.search_primary_key(multi_format_file_ref)

found primary key a
error: found 2 primary keys is same view file1
error: found 2 primary keys is same view file1


In [6]:
# extract views names
views_names = utils.get_view_names(multi_format_file_ref)

# rename columns names before join operation


pre_parsed_dataset_to_check, new_views_name = multiview.rename_variables_before_joining(pre_parsed_dataset_to_check, views_names,
                                                             primary_key)

In [7]:
# perform a join operation

# jointure operation (takesplace only if primary key has been specfied in foramt_file)
df_joined = multiview.join_multi_view_dataset(pre_parsed_dataset_to_check, primary_key, False)

In [8]:
df_joined

,a,e,time
0,48,98,2018-01-01 00:00:00
1,87,83,2018-01-01 01:00:00
2,46,73,2018-01-01 02:00:00
3,84,45,2018-01-01 03:00:00
4,94,84,2018-01-01 04:00:00
...,...,...,...
95,14,66,2018-01-04 23:00:00
96,91,81,2018-01-05 00:00:00
97,15,82,2018-01-05 01:00:00
98,51,18,2018-01-05 02:00:00


In [10]:
# convert joined dataset into multi view dataset
new_feature_name = { v: list(pre_parsed_dataset_to_check[v].columns) for v in views_names}
new_feature_name


multiview.create_multi_view_dataframe_from_dataframe(df_joined,new_feature_name, primary_key=primary_key)

length ['a', 'e', 'i', '0', '1', 'file1.time', 'pressure', 'sp02', 'a.1', 'gender', 'blood type', 'discrete', 'city', '2', 'file2.time', 'pH'] ['file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'contatct', 'contatct', 'file2', 'file2', 'file2']


views        file1                                                       \
feature_name     a   e   i      0      1           file1.time  pressure   
0               67  16  54   True  False  2018-01-03 04:00:00   0.98667   
1               42  96  69   True   True  2018-01-02 04:00:00  0.996889   
2               46   8  89  False   True  2018-01-01 09:00:00  0.777026   
3               29   6  77  False   True  2018-01-04 20:00:00  0.877527   
4               96  79  19  False   True  2018-01-04 09:00:00  0.447389   
..             ...  ..  ..    ...    ...                  ...       ...   
95              81  62  52   True   True  2018-01-02 13:00:00  0.953184   
96              16  49  30  False  False  2018-01-02 21:00:00  0.442283   
97              90  14  36  False  False  2018-01-02 06:00:00  0.988543   
98              91  10  69   True   True  2018-01-02 01:00:00  0.059791   
99              22  89  36  False   True  2018-01-03 22:00:00  0.939352   

views                                        contatct             file2  \
feature_name      sp02 a.1 gender blood type discrete       city      2   
0             0.676041   6  WOMAN          A     64.0      Lille  False   
1             0.864713  32    MAN         AB     26.0      Lille  False   
2             0.253232  64    MAN          A     61.0      Paris   True   
3             0.450811  73    MAN         AB     29.0      Paris  False   
4             0.639577  85  WOMAN          O     99.0      Lille   True   
..                 ...  ..    ...        ...      ...        ...    ...   
95            0.439232  34    MAN         AB      9.0      Paris  False   
96            0.600981  82    MAN        NaN     98.0  Marseille  False   
97            0.377624  35    MAN          B     21.0      Lille  False   
98             0.49036  42    MAN          B     42.0  Marseille  False   
99            0.143049  75    MAN          B      3.0      Paris   True   

views                                                 primary_key  
feature_name           file2.time        pH                  pkey  
0             2018-01-02 06:00:00       NaN  qpqorfhylu gmfjy bdj  
1             2018-01-01 00:00:00  0.023107  kkmjozalfyirgsire ui  
2             2018-01-02 10:00:00  0.587685  ezfasuuycdda foisjte  
3             2018-01-03 12:00:00  0.894073  faxiqkt xggzmwzoidbg  
4             2018-01-01 10:00:00  0.026831  znwhlj rwzdutnagwasy  
..                            ...       ...                   ...  
95            2018-01-02 05:00:00   0.78856  zeqhcikzdodus jn qjf  
96            2018-01-05 02:00:00  0.402979  iicthcvfmkajbvr gzir  
97            2018-01-01 12:00:00       NaN  ztjakcsk bhjoksdz lm  
98            2018-01-02 09:00:00  0.651801   sabunaa opt vpulnxj  
99            2018-01-03 02:00:00  0.751969  qmbexyexvgromrm admu  

[100 rows x 17 columns]

## Perform a data sanity check

In [9]:
# define some global varibales

MIN_NB_SAMPLES = 30  # TODO: specify it in data formt file (clinician should define it)

In [10]:
multi_format_file_ref

{'global_thresholds': {'min_nb_samples': None, 'min_nb_missing_samples': None},
 'file1': {'a': {'data_format': 'KEY',
   'data_type': 'DATETIME',
   'values': "<class 'numpy.datetime64'>",
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'e': {'data_format': 'KEY',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'time': {'data_format': 'KEY',
   'data_type': 'DATETIME',
   'values': "<class 'numpy.datetime64'>",
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None}}}

In [9]:
warning_report = WarningReportLogger(disclosure=3)

checker = PreProcessingChecker(multi_format_file_ref,
                               df_joined,
                               'multiview_dataFormat_file' ,
                               warning_report)

checker.update_views_features_name(new_views_name)
checker.check_all()



features names updated
INCORRECT_FORMAT_FILE
DATA_TYPE_MISMATCH
INCORRECT_DATA_TYPE
Variable named a should be a DATETIME variable, but it contains int64 type
OUTLIER_DETECTION_LOWER_BOUND
OUTLIER_DETECTION_UPPER_BOUND
INCORRECT_VALUES_CATEGORICAL_DATA
INCORRECT_DATETIME_DATA


## get report

In [10]:
report = checker.get_warning_logger()

number of warnings: 12
Number of error: 0


In [11]:
report

{'N_SAMPLES_BELOW_THRESHOLD': [{'view': 'global_thresholds',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'file1', 'success': True, 'feature': 'ALL', 'msg': 'Test passed'},
  {'view': 'contatct',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'file2', 'success': True, 'feature': 'ALL', 'msg': 'Test passed'}],
 'MISSING_FEATURE': [{'view': '',
   'success': False,
   'feature': 'min_nb_samples',
   'msg': 'MissingFeatureException: Feature min_nb_samples has not been found in dataset, but is needed for experiment'},
  {'view': '',
   'success': False,
   'feature': 'min_nb_missing_samples',
   'msg': 'MissingFeatureException: Feature min_nb_missing_samples has not been found in dataset, but is needed for experiment'}],
 'INCORRECT_FORMAT_FILE': [{'view': '',
   'success': True,
   'feature': 'a',
   'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'e', 'msg': 'Test passed'},
  {'view': '', 'success': True, '

In [26]:

report2 = {}

for k in report.keys():
    report2[k] = pd.DataFrame(report[k])

report2.pop('N_SAMPLES_BELOW_THRESHOLD')
report2.pop('INCORRECT_DATETIME_DATA')
multiview.create_multi_view_dataframe_from_dictionary(report2, ['test name', 'index'])

test name INCORRECT_FORMAT_FILE                                   \
index                      view success     feature          msg   
0                                  True           e  Test passed   
1                                  True           1  Test passed   
2                                  True           2  Test passed   
3                                  True        time  Test passed   
4                                  True    pressure  Test passed   
5                                  True         e.1  Test passed   
6                                  True      gender  Test passed   
7                                  True  blood type  Test passed   
8                                  True        pkey  Test passed   
9                                  True    discrete  Test passed   
10                                 True        city  Test passed   
11                                 True        pkey  Test passed   
12                                 True           1  Test passed   
13                                 True        time  Test passed   
14                                 True          pH  Test passed   
15                                 True        pkey  Test passed   

test name DATA_TYPE_MISMATCH                                   \
index                   view success     feature          msg   
0                               True           e  Test passed   
1                               True           1  Test passed   
2                               True           2  Test passed   
3                               True        time  Test passed   
4                               True    pressure  Test passed   
5                               True         e.1  Test passed   
6                               True      gender  Test passed   
7                               True  blood type  Test passed   
8                               True        pkey  Test passed   
9                               True    discrete  Test passed   
10                              True        city  Test passed   
11                              True        pkey  Test passed   
12                              True           1  Test passed   
13                              True        time  Test passed   
14                              True          pH  Test passed   
15                              True        pkey  Test passed   

test name INCORRECT_DATA_TYPE          ... OUTLIER_DETECTION_LOWER_BOUND  \
index                    view success  ...                       feature   
0                                True  ...                             e   
1                                True  ...                             1   
2                                True  ...                             2   
3                                True  ...                          time   
4                                True  ...                      pressure   
5                                True  ...                           e.1   
6                                True  ...                        gender   
7                                True  ...                    blood type   
8                                True  ...                          pkey   
9                                True  ...                      discrete   
10                               True  ...                          city   
11                               True  ...                          pkey   
12                               True  ...                             1   
13                               True  ...                          time   
14                               True  ...                            pH   
15                               True  ...                          pkey   

test name               OUTLIER_DETECTION_UPPER_BOUND                      \
index               msg                          view success     feature   
0          Test skipped                                  None           e   
1   

In [13]:
report

{'N_SAMPLES_BELOW_THRESHOLD': [{'view': 'file1',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'contatct',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'file2', 'success': True, 'feature': 'ALL', 'msg': 'Test passed'}],
 'INCORRECT_FORMAT_FILE': [{'view': '',
   'success': True,
   'feature': 'e',
   'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': '1', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': '2', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'time', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'pressure', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'e.1', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'gender', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'blood type', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'pkey', 'msg': 'Test passed'},
  {'view': ''

# raise collected error

This will raises all collected error found in the warning logger

In [27]:
checker.get_warning_logger.raise_exception()

DataSanityCheckException: MissingDataException: Variable pH must not have missing data, but some were found